# 💳 Personal Finances Expense Tracker
This is a simple finance program I built using Python to track my credit card expenses. The goal is to calculate and visualize total monthly spending grouped by category. I can track where my money goes over time.
The data comes from my personal credit card statements, covering the period from January 2025 to the end of March 2025. This timeframe includes a major milestone, my graduation 🎓 , and also marks the beginning of my job hunt journey. Since I’m in a transitional phase, I thought it would be the perfect time to monitor my spending habits and be more mindful of my finances...
Until my next big adventure begins 🌍✨


In [85]:
#Import packages

import pandas as pd
import numpy as np

#Build dashboard
import panel as pn
pn.extension('tabulator')
#plots
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')
import re

In [86]:
#data from my credit card
df = pd.read_csv("credit.csv")
df.head() #look at the data

,Date,Description,Amount
0,"Mar 30, 2025","TANYA'S NF KINGSTON 75, KINGSTON",14.25
1,"Mar 30, 2025","POPARIDE, VANCOUVER",18.41
2,"Mar 29, 2025","JOHN'S VARIETY, BELLEVILLE",11.23
3,"Mar 29, 2025","FLIXBUS INC, Dallas",17.98
4,"Mar 28, 2025","LCBO/RAO #0402, BELLEVILLE",9.45


In [87]:
df=df[['Description','Amount','Date']]#only keep required codes
df['Description']=df['Description'].map(str.lower)# lower case the names 
df['Category']='Unexpected'#until defining the categories keep everything as unexpected
df.head()

,Description,Amount,Date,Category
0,"tanya's nf kingston 75, kingston",14.25,"Mar 30, 2025",Unexpected
1,"poparide, vancouver",18.41,"Mar 30, 2025",Unexpected
2,"john's variety, belleville",11.23,"Mar 29, 2025",Unexpected
3,"flixbus inc, dallas",17.98,"Mar 29, 2025",Unexpected
4,"lcbo/rao #0402, belleville",9.45,"Mar 28, 2025",Unexpected


# 🗂️ Define Categories

To track my spending meaningfully, I analyzed my credit card records to identify the most frequently visited stores each month and assigned them to simple, intuitive categories.

Since I’m currently in the transition phase of applying for a work visa, some expenses, such as visa application fees and an English language test, are grouped under a special category I called “Paperwork”.

One of my main financial goals during this time is to minimize restaurant spending and subscriptions. I'm even making my own coffee at home ☕, so I’ve separated out those expenses to monitor them more closely.

By keeping the categories clear and relevant to my current situation, this tracker helps me stay more intentional about how I spend.

In [88]:
#define categories 

#Groceries
df['Category']=np.where(df['Description'].str.contains('food basics|costco|dollarama|loblaws|lcbo|nf|variety|freshco|bulk|metro|wal-mart'),'Groceries',df['Category'])

#Health
df['Category']=np.where(df['Description'].str.contains('shoppers|rexall|drug|coinamatic'),'Health',df['Category'])

#PaperWork
df['Category']=np.where(df['Description'].str.contains('paragon|immigration|queen|service'),'PaperWork',df['Category'])

#Shopping
df['Category']=np.where(df['Description'].str.contains('amazon|amzn'),'Shopping',df['Category'])

#Trasport
df['Category']=np.where(df['Description'].str.contains('flixbus|poparide|uber|breakaway|esso|honk|presto|park'),'Transport',df['Category'])

#Subs
df['Category']=np.where(df['Description'].str.contains('netflix|spotify|linkedin|openai|virgin|mobile|apple|disney'),'Subs',df['Category'])

#Pet
df['Category']=np.where(df['Description'].str.contains('trupanion'),'Pet',df['Category'])

#Resturant
df['Category']=np.where(df['Description'].str.contains('brewing|deli|piper|pizza|denny|flip|thai|elephant|stacked|shuroop|korean|wing'),'Restaurant',df['Category'])
#Coffee
df['Category']=np.where(df['Description'].str.contains('coffee|starbucks|tim|corry|grocery|holiday'),'Coffee',df['Category'])


In [89]:
#Clean hidden characters and whitespace
df['Date'] = df['Date'].astype(str).apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x).strip()) # make sure there are no special characters

# Convert to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')

#Extract Month and Year
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year           
pd.options.display.max_rows=999

In [90]:
#to track what are unexpected 
unexpected_rows = df[df['Category'] == 'Unexpected']

In [91]:
#Get the month and year 
latest_month=df['Month'].max()
latest_year=df['Year'].max()

#filter the data for monthly transactions:

last_month_trans=df[(df['Month']==latest_month) & (df['Year']==latest_year)]

# 📊 Aggregating Expenses by Category For Last Month
The code below calculates the total expenses for each defined category.
To make the output easier to read and interpret, the amounts are rounded to the nearest whole number.


In [92]:
last_month_trans=last_month_trans.groupby('Category')['Amount'].sum().reset_index()

last_month_trans['Amount']=last_month_trans['Amount'].astype('str')
last_month_trans['Amount']=last_month_trans['Amount'].str.replace('-','')
last_month_trans['Amount']=last_month_trans['Amount'].astype('float')

#sort amount
last_month_trans=last_month_trans.sort_values(by='Amount',ascending=False)

#round
last_month_trans['Amount']=last_month_trans['Amount'].round().astype(int)
last_month_trans

,Category,Amount
3,PaperWork,346
1,Groceries,332
8,Unexpected,253
6,Subs,160
7,Transport,123
5,Restaurant,108
4,Pet,68
0,Coffee,12
2,Health,8


In [93]:
last_month_trans_total=last_month_trans['Amount'].sum()
last_month_trans_total

1410

# 💰 Calculating Monthly Savings

The function below calculates monthly savings by accounting for:
Income, Fixed expenses (such as rent), Variable expenses from the credit card (which fluctuate month to month).
This breakdown helps provide a clear picture of how much I’m saving each month during this transitional period. 

In [94]:
def income_diff(money):
    income=float(income_wdj.value)
    fixed_expenses=float(fixed_expenses_wdj.value)
    monthly_expenses=float(monthly_expences_wdj.value)
    difference=income-fixed_expenses-monthly_expenses
    differece_wdj.value=str(difference)

income_wdj=pn.widgets.TextInput(name='Income',value='0')
fixed_expenses_wdj=pn.widgets.TextInput(name='Fixed Expenses', value='0')
monthly_expences_wdj=pn.widgets.TextInput(name='Monthly Expenses',value=str(last_month_trans_total))
differece_wdj=pn.widgets.TextInput(name='Saving',value='0')

income_wdj.param.watch(income_diff,'value')
fixed_expenses_wdj.param.watch(income_diff,'value')
monthly_expences_wdj.param.watch(income_diff,'value')

#pn.Row(income_wdj,fixed_expenses_wdj,monthly_expences_wdj,differece_wdj).show()



Watcher(inst=TextInput(name='Monthly Expenses', value='1410'), cls=<class 'panel.widgets.input.TextInput'>, fn=<function income_diff at 0x12c6a9da0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [95]:
last_month_trans_chart=last_month_trans.hvplot.bar(
    x='Category',
    y='Amount',
    height=350,
    width=1050,
    title='March Transactions',
    ylim=(0,500)
)
last_month_trans_chart

:Bars   [Category]   (Amount)

# 📆  Monthly Comparison by Category

The goal here is to prepare the original data so that each category is represented across all months. After that, group the data by category and month, which allows us to create an interactive Panel dashboard.

This enables an easy comparison of how spending in each category changes from month to month, helping identify trends and areas for improvement over time.

In [96]:
df['Date']=pd.to_datetime(df['Date'])
df['Month-Year']=df['Date'].dt.to_period('M')
Monthly_trans_by_cat=df.groupby(['Month-Year','Category'])['Amount'].sum().reset_index()

Monthly_trans_by_cat['Amount']=Monthly_trans_by_cat['Amount'].astype('str')
Monthly_trans_by_cat['Amount']=Monthly_trans_by_cat['Amount'].str.replace('-','')
Monthly_trans_by_cat['Amount']=Monthly_trans_by_cat['Amount'].astype('float')
#Monthly_trans_by_cat=Monthly_trans_by_cat[Monthly_trans_by_cat['Category']].str.contains('')

Monthly_trans_by_cat=Monthly_trans_by_cat.sort_values(by='Amount',ascending=False)
Monthly_trans_by_cat['Amount']=Monthly_trans_by_cat['Amount'].round().astype(int)
Monthly_trans_by_cat['Month-Year']=Monthly_trans_by_cat['Month-Year'].astype(str)
Monthly_trans_by_cat=Monthly_trans_by_cat.rename(columns={'Amount':'Amount '})

Monthly_trans_by_cat

,Month-Year,Category,Amount
17,2025-02,Unexpected,507
1,2025-01,Groceries,451
10,2025-02,Groceries,368
21,2025-03,PaperWork,346
14,2025-02,Shopping,338
19,2025-03,Groceries,332
26,2025-03,Unexpected,253
5,2025-01,Restaurant,173
24,2025-03,Subs,160
6,2025-01,Shopping,149


In [97]:
#define panel

select_cat=pn.widgets.Select(name='Select Category',options=[
    'All',
    'Groceries',
    'Health',
    'PaperWork',
    'Shopping',
    'Trasport',
    'Subs',
    'Pet',
    'Restaurant',
    'Coffee',
    'Unexpected'
    ])

select_cat




Select(options=['All', 'Groceries', ...], value='All')

In [98]:
def plot_expenses(cat):
    if cat=='All':
        plot_df=Monthly_trans_by_cat.groupby('Month-Year').sum()
    else:
        plot_df=Monthly_trans_by_cat[Monthly_trans_by_cat['Category']==cat].groupby('Month-Year').sum()
    plot= plot_df.hvplot.bar(x='Month-Year',y='Amount ')
    return plot

@pn.depends(select_cat.param.value)
def update_plot(cat):
    plot=plot_expenses(cat)
    return plot

Monthly_trans_by_cat_chart=pn.Row(select_cat,update_plot)
Monthly_trans_by_cat_chart[1].width=700

Monthly_trans_by_cat_chart

Row
    [0] Select(options=['All', 'Groceries', ...], value='All')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=700)

# 📋 Summary of Expenses by Category

This section provides a detailed summary of expenses for each category.

Using the dropdown selector, we can choose a specific category or select "All" to view the full breakdown across all categories.
This makes it easier to explore and compare how different types of expenses contribute to the overall monthly spending.


In [103]:
df=df[['Date','Category','Description','Amount']]
df['Amount']=df['Amount'].astype('str')
df['Amount']=df['Amount'].str.replace('-','')
df['Amount']=df['Amount'].astype('float')

df['Amount']=df['Amount'].round().astype(int)
df.head()

,Date,Category,Description,Amount
0,2025-03-30,Groceries,"tanya's nf kingston 75, kingston",14
1,2025-03-30,Transport,"poparide, vancouver",18
2,2025-03-29,Groceries,"john's variety, belleville",11
3,2025-03-29,Transport,"flixbus inc, dallas",18
4,2025-03-28,Groceries,"lcbo/rao #0402, belleville",9


In [105]:
def filter_df(cat):
    if cat=='All':
        return df
    return df[df['Category']==cat]

summary_table=pn.widgets.DataFrame(filter_df('All'),height=400,width=450)

def update_summary_table(event):
    summary_table.value=filter_df(event.new)

select_cat.param.watch(update_summary_table,'value')

summary_table


DataFrame(height=400, sizing_mode='fixed', value=          Date    Category..., width=450)

# 🐷 Let's See How to Save More in Our Piggy Bank!

This panel displays the category-wise breakdown of March expenses.

It’s clear that the majority of my spending this month went toward paperwork, as I’m currently preparing visa applications and related documents.

Sadly, my beloved Grandma passed away during this time. Since I couldn't travel back to my hometown, I sent flowers — this expense is categorized under "Unexpected".

On a positive note, when comparing February and March, I’ve noticeably reduced my spending on restaurants, which means I’m on the right track with my budgeting goals! 🍽️✅

I’m excited to add next month’s transactionsand see if I’m able to save more. For now, I’m mostly relying on my savings due to all the paperwork and those unforeseen costs, but progress is progress!


In [107]:
template=pn.template.FastListTemplate(
    title='Personal Finance',
    sidebar=[
        pn.pane.Markdown('## *From Daily Spending to Dream Adventures!*'),
        pn.pane.PNG('piggy.png',sizing_mode='scale_both'),
        pn.pane.Markdown(''),
        pn.pane.Markdown(''),
        select_cat
    ],
    main=[
        pn.Row(income_wdj,fixed_expenses_wdj,monthly_expences_wdj,differece_wdj,width=1050),
        pn.Row(last_month_trans_chart,height=340),
        pn.GridBox(
            Monthly_trans_by_cat_chart[1],
            summary_table,
            ncols=2,
            min_width=400,
            align='start',
            sizing_mode = 'stretch_width'

        )
    ]
    
)
template.show()

Launching server at http://localhost:58642


# ✅ Conclusion

We’ve seen where most of the money goes, and identified areas where I can start saving more, all in preparation for my next big adventure! 🚀💸